In [1]:
import numpy as np
import pandas as pd
import re
import typing as t
import nltk
from nltk.tokenize import word_tokenize
# from nltk.stem import SnowballStemmer
from pymorphy3 import MorphAnalyzer

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [2]:
df = pd.read_csv('../storage/Petitions.csv', encoding='utf-8', sep=',')
df

,id,public_petition_text,reason_category
0,3168490,снег на дороге,Благоустройство
1,3219678,очистить кабельный киоск от рекламы,Благоустройство
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД
4,3336285,Граффити,Благоустройство
...,...,...,...
59884,3128111,прошу закрасить граффити,Благоустройство
59885,3276713,Прошу вас отремонтировать пешеходную дорожку,Благоустройство
59886,3274663,Необходимо демонтировать незаконную рекламную ...,Незаконная информационная и (или) рекламная ко...
59887,3359308,Очень гремит на ветру металлическая часть окна...,Кровля


In [3]:
corpus = df['public_petition_text'].values.tolist()
mega_document = ' '.join(df['public_petition_text'].values)

In [4]:
len(mega_document)

5837511

In [5]:
corpus[:25]

['снег на дороге',
 'очистить кабельный киоск от рекламы',
 'Просим убрать все деревья и кустарники, которые вышли за пределы газона на пешеходную зону, начиная от подъезда №13 до подъезда №15 (фасад дома со стороны ул. Наличной).',
 'Неудовлетворительное состояние парадной - надписи на двери 2 этажа',
 'Граффити',
 'Необходимо проверить законность установки вывески на фасаде МКД по адресу проспект Непокорённых 74. В случае, если вывеска установлена незаконно её необходимо демонтировать',
 'Уборка не производится, на лестнице очень грязно. На всех этажах, вплоть до 5-го.\r\nЗвонок в ЖКС#2 5.04.2021 не дал результатов.',
 'Мусор',
 'Отсутствует освещение на лестничной площадке между 6 и 7 этажами в парадной № 2',
 'Зачем было делать благоустройство, если никто не убирает мусор??? И так ежедневно!',
 'Просьба закрасить',
 'Реклама на заборе.',
 'Снег с тротуаре не убран',
 'Проблема с регулярным вывозом мусора',
 'Рисунки',
 'Пожалуйста, удалите бетонный обрубок с ржавой арматурой с газо

In [5]:
def apply_re(pattern: str, reply_on: str, corpus: t.Sequence[str]) -> t.MutableSequence[str]:
    return list(map(lambda x: re.sub(pattern, reply_on, x), corpus))

In [6]:
corpus = apply_re('(<[^>]+>|\s)', ' ', corpus)
corpus = apply_re('ё', 'е', corpus)
corpus = apply_re(' ул(?=[\.\d ])', ' улица ', corpus)
corpus = apply_re(' (д|дом)(?=[\.\d ])', ' дом ', corpus)
corpus = apply_re(' (к|корп|корпус|кор)(?=[\.\d ])', ' корпус ', corpus)
corpus = apply_re(' (к|кв)(?=[\.\d])', ' квартира ', corpus)
corpus = apply_re('[^{а-яА-Яa-zA-Z0-9}]', ' ', corpus)
corpus = apply_re('(\d|( . ))', ' ', corpus)
corpus = apply_re('\s+', ' ', corpus)

corpus[:25]

['снег на дороге',
 'очистить кабельный киоск от рекламы',
 'Просим убрать все деревья кустарники которые вышли за пределы газона на пешеходную зону начиная от подъезда до подъезда фасад дома со стороны улица Наличной ',
 'Неудовлетворительное состояние парадной надписи на двери этажа',
 'Граффити',
 'Необходимо проверить законность установки вывески на фасаде МКД по адресу проспект Непокоренных случае если вывеска установлена незаконно ее необходимо демонтировать',
 'Уборка не производится на лестнице очень грязно На всех этажах вплоть до го Звонок ЖКС не дал результатов ',
 'Мусор',
 'Отсутствует освещение на лестничной площадке между и этажами парадной ',
 'Зачем было делать благоустройство если никто не убирает мусор так ежедневно ',
 'Просьба закрасить',
 'Реклама на заборе ',
 'Снег тротуаре не убран',
 'Проблема регулярным вывозом мусора',
 'Рисунки',
 'Пожалуйста удалите бетонный обрубок ржавой арматурой газона перед домом Адрес улица Тухачевского дом корпус ',
 'Мусор на асфал

In [8]:
corpus[:100]

['снег на дороге',
 'очистить кабельный киоск от рекламы',
 'Просим убрать все деревья кустарники которые вышли за пределы газона на пешеходную зону начиная от подъезда до подъезда фасад дома со стороны улица Наличной ',
 'Неудовлетворительное состояние парадной надписи на двери этажа',
 'Граффити',
 'Необходимо проверить законность установки вывески на фасаде МКД по адресу проспект Непокоренных случае если вывеска установлена незаконно ее необходимо демонтировать',
 'Уборка не производится на лестнице очень грязно На всех этажах вплоть до го Звонок ЖКС не дал результатов ',
 'Мусор',
 'Отсутствует освещение на лестничной площадке между и этажами парадной ',
 'Зачем было делать благоустройство если никто не убирает мусор так ежедневно ',
 'Просьба закрасить',
 'Реклама на заборе ',
 'Снег тротуаре не убран',
 'Проблема регулярным вывозом мусора',
 'Рисунки',
 'Пожалуйста удалите бетонный обрубок ржавой арматурой газона перед домом Адрес улица Тухачевского дом корпус ',
 'Мусор на асфал

In [9]:
# stemmer = SnowballStemmer('russian')
# documents = [
#     list(map(stemmer.stem, doc))
#     for doc in documents
# ]
# documents[:5]

In [7]:
morph = MorphAnalyzer()
stop_words = stopwords.words('russian')

documents = [
    [
        word_norm for word in word_tokenize(doc, language='russian')
        if (word_norm := morph.normal_forms(word)[0]) not in stop_words
    ] for doc in corpus
]

# documents = [
#     doc for document in corpus
#     if (doc := [
#         word_norm for word in word_tokenize(document, language='russian')
#         if (word_norm := morph.normal_forms(word)[0]) not in stop_words
#     ])
# ]

documents[:5]

[['снег', 'дорога'],
 ['очистить', 'кабельный', 'киоск', 'реклама'],
 ['просить',
  'убрать',
  'всё',
  'дерево',
  'кустарник',
  'который',
  'выйти',
  'предел',
  'газон',
  'пешеходный',
  'зона',
  'начинать',
  'подъезд',
  'подъезд',
  'фасад',
  'дом',
  'сторона',
  'улица',
  'наличный'],
 ['неудовлетворительный', 'состояние', 'парадный', 'надпись', 'дверь', 'этаж'],
 ['граффити']]

In [8]:
len(documents)

59889

In [15]:
idx2word = dict(enumerate(set().union(*documents)))
word2idx = {v: k for k, v in idx2word.items()}
voc_len = len(idx2word)
voc_len

16774

In [17]:
context = np.zeros((voc_len, voc_len))

def add_context(word1: str, word2: str, context: np.array, word2idx: t.Mapping[str, int]) -> None:
    context[word2idx[word1], word2idx[word2]] += 1

In [18]:
corpus = documents
window_size = 2

for _, doc in enumerate(corpus):
    for i, word1 in enumerate(doc):
        for word2 in doc[i - window_size: i + window_size]:
            add_context(word1, word2, context, word2idx)
    if not _ % 100:
        print(_)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [19]:
context.shape

(16774, 16774)

In [21]:
print(np.unique(context[123]))
print(context[123])

[0. 1.]
[0. 0. 0. ... 0. 0. 0.]


In [18]:
from sklearn.decomposition import PCA

In [19]:
pd.DataFrame(data=idx2word, columns=['word_idx', 'word']).to_csv('../storage/corpus.tsv', sep='\t', index=False)

In [20]:
for n in (3, 100, 200):
    n_pca = PCA(n_components=n).fit(context)
    print(f'[INFO] {n} components save {sum(n_pca.explained_variance_ratio_):.3%} information.')
    np.savetxt(f'../storage/ctx_{n}.tsv', n_pca.transform(context), delimiter='\t')

[INFO] 3 components save 32.174% information.
[INFO] 100 components save 89.464% information.
[INFO] 200 components save 94.858% information.


In [21]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

torch.cuda_version, torch.cuda.is_available(), torch.cuda.get_device_name(torch.cuda.current_device())

('11.8', True, 'NVIDIA GeForce GTX 1650')

In [22]:
class CBoW(nn.Module):
    def __init__(self, vocab_size, embedding_dim) -> None:
        super(CBoW, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        # self.flatten = nn.Flatten()
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, inputs):
        y = self.embedding(inputs).mean(1)
        # y = self.flatten(y)
        y = self.linear(y)
        return y

In [24]:
# Либо (контекст, центральное) заменять на (контекстное, центральное)[..2*window_size]

X_ = []
y_ = []
max_context_size = window_size * 2
max_idx = len(idx2word)
for doc in corpus:
    doc_size = len(doc)

    for i, word in enumerate(doc):
        if doc_size == 1:
            word_context = [word]
        else:
            word_context = doc[max(0, i - window_size):i] + doc[i + 1: i + window_size + 1]

        X_.append([word2idx[w] for w in word_context])
        y_.append(word2idx[word])

        if (diff := (max_context_size - len(word_context))) > 0:
            X_[-1] += [max_idx] * diff

list(zip(X_[:5], y_[:5]))

[([15035, 16774, 16774, 16774], 10925),
 ([10925, 16774, 16774, 16774], 15035),
 ([530, 10132, 16774, 16774], 15944),
 ([15944, 10132, 15328, 16774], 530),
 ([15944, 530, 15328, 16774], 10132)]

In [25]:
X = torch.LongTensor(X_).cuda()
y = torch.LongTensor(y_).cuda()

train_ds = TensorDataset(X, y)
train_dl = DataLoader(train_ds, batch_size=1024, shuffle=True)

In [27]:
lr = 1e-3
epochs = 15
loss = nn.CrossEntropyLoss()
cbow = CBoW(voc_len + 1, 100)  # из-за символа, уравнивающего размер контекста
optimizer = torch.optim.Adam(cbow.parameters(), lr=lr)

cbow = cbow.cuda()
all(param.is_cuda for param in cbow.parameters())

True

In [28]:
for epoch in range(epochs):
    print(f'[INFO] Epoch {epoch + 1}...')
    for x_b, y_b in train_dl:
        outputs = cbow(x_b)
        loss_value = loss(outputs, y_b)
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()

[INFO] Epoch 1...
[INFO] Epoch 2...
[INFO] Epoch 3...
[INFO] Epoch 4...
[INFO] Epoch 5...
[INFO] Epoch 6...
[INFO] Epoch 7...
[INFO] Epoch 8...
[INFO] Epoch 9...
[INFO] Epoch 10...
[INFO] Epoch 11...
[INFO] Epoch 12...
[INFO] Epoch 13...
[INFO] Epoch 14...
[INFO] Epoch 15...


In [29]:
X_test = torch.LongTensor([[word2idx[w] for w in ['парадный', 'надпись', 'дверь']]]).cuda()
y_pred = cbow(X_test)
y_pred

tensor([[-6.3617, -3.0562,  3.4435,  ..., -7.4562, -5.6841, -9.8353]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [30]:
len(y_pred[0])

16775

In [31]:
weights = cbow.linear.weight.cpu().detach().numpy()
weights

array([[ 0.03045158,  0.14411707,  0.251575  , ...,  0.28689522,
         0.0091321 ,  0.08549227],
       [-0.1453822 , -0.5039491 , -0.25046924, ..., -0.37252963,
         0.28716987, -0.51638025],
       [-0.16754252, -0.09433295, -0.02163976, ...,  0.30215096,
         0.00655744,  0.03419391],
       ...,
       [-0.02054624,  0.00821832, -0.8629965 , ...,  0.23764642,
        -0.19747852, -0.29738095],
       [-0.05664107, -0.41907287, -0.19835967, ...,  0.3996541 ,
         0.06972168, -0.0640636 ],
       [-0.4346393 , -0.6197143 , -0.7460473 , ...,  0.37999013,
        -0.6601136 , -0.74064904]], dtype=float32)

In [32]:
weights.shape

(16775, 100)

In [33]:
weights[-1]

array([-0.4346393 , -0.6197143 , -0.7460473 ,  0.4892927 , -0.4701231 ,
       -0.3384981 ,  0.4196527 ,  0.5402737 ,  0.3611851 , -0.40654862,
       -0.38621452, -0.37819946, -0.4466699 ,  0.39720267,  0.3904711 ,
        0.66323   , -0.18337679, -0.288704  ,  0.5119338 , -0.19981484,
        0.33007142,  0.58261067, -0.4996446 , -0.42386058, -0.68418926,
        0.38299182,  0.42624646, -0.5292376 ,  0.751342  ,  0.6044369 ,
        0.4330175 ,  0.4707195 ,  0.4143036 ,  0.6181003 , -0.45878768,
        0.46123642,  0.38686603, -0.59334666,  0.34861523, -0.1620569 ,
        0.32485184,  0.15930986,  0.5679761 , -0.07899619,  0.53152764,
       -0.42520255, -0.4572836 ,  0.24291544,  0.40005684,  0.34699294,
       -0.7596886 ,  0.4058983 , -0.4081963 ,  0.18856567, -0.6334694 ,
       -0.46348467,  0.38326102, -0.11798025, -0.57196337,  0.5388918 ,
       -0.40943962,  0.37202916, -0.41553733, -0.46615925,  0.47528467,
        0.5965829 , -0.46929246,  0.6577087 , -0.5057732 , -0.15

In [34]:
idx2word[212], weights[212]

('отвратительный',
 array([-0.46850115, -0.34173474,  0.4448694 ,  0.2954142 ,  0.14294711,
         0.01440961, -0.23649351,  0.07113781, -0.30113557, -0.22905686,
        -0.14381357, -0.42626005,  0.2506435 , -0.13032259,  0.18500149,
        -0.2593535 , -0.09758908, -0.43022814, -0.15877283,  0.2691209 ,
         0.1999621 ,  0.04145608,  0.14591388,  0.01316757, -0.09866389,
         0.01275306, -0.12277415,  0.37809223,  0.1761022 , -0.02774519,
        -0.17622185, -0.09820765,  0.12589379, -0.34053385, -0.16310522,
        -0.03556273, -0.22726302,  0.38775897,  0.5272783 ,  0.19972606,
        -0.36405465,  0.25169083,  0.10996608, -0.28111297, -0.20741162,
        -0.37517762,  0.0221365 ,  0.11310711, -0.12742905,  0.06191311,
        -0.27147087,  0.1739181 , -0.3410638 , -0.07510699,  0.03732627,
         0.22943847,  0.37419492,  0.12276009, -0.08895335,  0.07061756,
         0.15123478,  0.1338614 ,  0.01884235,  0.03334788, -0.48767638,
        -0.05093409,  0.19534457

In [37]:
n = 55
cbow_pca = PCA(n)
np.savetxt(f'../storage/cbow_{n}.tsv', cbow_pca.fit_transform(weights), delimiter='\t')
sum(cbow_pca.explained_variance_ratio_)

0.652122993953526